In [2]:
from pyspark.sql import SparkSession
#build next stop id dataframe
from pyspark.sql import Window, functions as F

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("CompareShortTrips") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/30 14:42:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def haversine(lat1_col, lon1_col, lat2_col, lon2_col):
    """
    Calculate the great circle distance in meters between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lat1 = F.radians(lat1_col)
    lon1 = F.radians(lon1_col)
    lat2 = F.radians(lat2_col)
    lon2 = F.radians(lon2_col)
    
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = F.sin(dlat/2)**2 + F.cos(lat1) * F.cos(lat2) * F.sin(dlon/2)**2
    c = 2 * F.asin(F.sqrt(a)) 
    r = 6371000  # Radius of earth in meters. Use 3956 for miles. Determines return value units.
    return c * r

def prepare_journeys(journeys,stop_times):
    alighting_stop_window = Window.partitionBy("CARD_ID").orderBy(F.col("DATETIME").asc())
    journeys = journeys.join(stop_times.dropDuplicates(),on = ["LINE_ID","STOP_ID"],how = "left")
    journeys = journeys.dropDuplicates(["CARD_ID","JOURNEY_ID","DATETIME","EVENT"])
    journeys = journeys.withColumn(
        "EVENT_NEXT_NEXT",
        F.lead(F.col("EVENT"),2).over(alighting_stop_window)
        ).withColumn(
        "EVENT_TYPE_NEXT_NEXT",
        F.lead(F.col("EVENT_TYPE"),2).over(alighting_stop_window)
        )
    journeys = journeys.withColumn(
        "STOP_ID_NEXT",
        F.lead(F.col("STOP_ID"),1).over(alighting_stop_window)
        ).withColumn(
        "STOP_LAT_NEXT",
        F.lead(F.col("STOP_LAT"),1).over(alighting_stop_window)
        ).withColumn(
        "STOP_LON_NEXT",
        F.lead(F.col("STOP_LON"),1).over(alighting_stop_window)
        ).withColumn(
        "STOP_ID_NEXT_NEXT",
        F.lead(F.col("STOP_ID"),2).over(alighting_stop_window)
        ).withColumn(
        "STOP_LAT_NEXT_NEXT",
        F.lead(F.col("STOP_LAT"),2).over(alighting_stop_window)
        ).withColumn(
        "STOP_LON_NEXT_NEXT",
        F.lead(F.col("STOP_LON"),2).over(alighting_stop_window)
        ).withColumn(
        "DATETIME_NEXT_NEXT",
        F.lead(F.col("DATETIME"),2).over(alighting_stop_window)
        ).withColumn(
            "CONFIDENCE_NEXT",
            F.lead(F.col("CONFIDENCE"),1).over(alighting_stop_window)
        )
    journeys = journeys.withColumn("DISTANCE_TO_NEXT_STOP",haversine(
        F.col("STOP_LAT"),
        F.col("STOP_LON"),
        F.col("STOP_LAT_NEXT"),
        F.col("STOP_LON_NEXT"))
    )
    journeys = journeys.withColumn("DISTANCE_TO_NEXT_NEXT_STOP",haversine(
        F.col("STOP_LAT"),
        F.col("STOP_LON"),
        F.col("STOP_LAT_NEXT_NEXT"),
        F.col("STOP_LON_NEXT_NEXT"))
    )
    journeys = journeys.withColumn("ALIGHTING_DISTANCE_TO_NEXT_STOP",haversine(
        F.col("STOP_LAT_NEXT"),
        F.col("STOP_LON_NEXT"),
        F.col("STOP_LAT_NEXT_NEXT"),
        F.col("STOP_LON_NEXT_NEXT"))
    )
    journeys = journeys.withColumn(
        "TIME_TO_NEXT_ORIGIN",
        F.datediff(F.col("DATETIME_NEXT_NEXT"),
                F.col("DATETIME")
                ))
    return journeys

In [4]:
stop_times = spark.read.parquet("../data/02_intermediate/stop_times_avl/stop_times")
stop_times = stop_times.dropDuplicates(["TRIP_ID","STOP_SEQUENCE"])
next_stop_window = Window.partitionBy("TRIP_ID").orderBy(F.col("STOP_SEQUENCE").asc())
stop_times = stop_times.withColumn(
    "STOP_ID_NEXT",
    F.lead(F.col("STOP_ID"),1).over(next_stop_window)
    )
stop_times = stop_times.select(
    F.col("ROUTE_ID_OLD").alias("LINE_ID"),
    F.col("STOP_ID"),
    F.col("STOP_ID_NEXT").alias("TRIP_STOP_ID_NEXT"),
    F.col("DIRECTION_ID"),
).dropDuplicates(["LINE_ID","STOP_ID","DIRECTION_ID"])
stop_times = stop_times.withColumn(
    "IS_MAX",
    F.when(F.col("LINE_ID").isin([200,190,290,90,100]),"Y").otherwise("N")
    ).cache()
fixed_journeys = spark.read.parquet("../data/03_primary/rider_events_partitioned")

In [12]:
#get journey distance profiles
journey_window = Window.partitionBy("CARD_ID").orderBy(F.col("DATETIME").asc())
origin_distinations = fixed_journeys.filter(F.col("EVENT_TYPE").isin(["ORIGIN","DESTINATION"]))
origin_distinations = origin_distinations.select(
    *[F.col(col).alias(f"{col}_ORIGIN") for col in origin_distinations.columns],
    *[F.lead(F.col(col),1).over(journey_window).alias(f"{col}_DESTINATION") for col in origin_distinations.columns],
    *[F.lead(F.col(col),2).over(journey_window).alias(f"{col}_NEXT_ORIGIN") for col in origin_distinations.columns]
    ).filter(
        F.col("EVENT_TYPE_ORIGIN")=="ORIGIN"
    ).filter(
        F.col("EVENT_TYPE_DESTINATION")=="DESTINATION"
    ).filter(
        F.col("EVENT_TYPE_NEXT_ORIGIN")=="ORIGIN"
    )
origin_distinations.show()

+--------------------+--------------------------------+-------------------+--------------+--------------------+--------------+-----------------+-------------------+-------------------+------------+-----------------+-----------------+--------------+-------------------------+----------------------+-------------------------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+------------------------+-----------------+----------------------+----------------------+-------------------+------------------------------+----------------------+-------------------------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+------------------------+-----------------+----------------------+----------------------+-------------------+------------------------------+
|   JOURNEY_ID_ORIGIN|FARE_CATEGORY_DESCRIPTION_ORIGIN|    DATETIME_ORIGIN|

In [31]:
journeys = fixed_journeys.filter(F.col("LINE_ID")==6).filter(F.col("STOP_ID")==1026).select("JOURNEY_ID").toPandas()["JOURNEY_ID"].values.tolist()
window = Window.partitionBy("JOURNEY_ID").orderBy(F.col("DATETIME"))
check_bad_journeys = fixed_journeys.filter(
    F.col("JOURNEY_ID").isin(journeys)
    ).select(
        "JOURNEY_ID",
        "STOP_ID",
        "EVENT",
        "EVENT_TYPE",
        F.col("DATETIME").cast("int"),"CARD_ID",
        F.lead(F.col("DATETIME").cast("int"),1).over(window).alias("DATETIME_NEXT"),
        F.lead(F.col("EVENT_TYPE"),1).over(window).alias("EVENT_TYPE_NEXT"),
        ).toPandas()

In [35]:
check_bad_journeys["TRAVEL_TIME"] = check_bad_journeys["DATETIME_NEXT"]-check_bad_journeys["DATETIME"]
check_bad_journeys[
    (check_bad_journeys["EVENT_TYPE"]=="ORIGIN")&
    (check_bad_journeys["EVENT_TYPE_NEXT"]=="DESTINATION")
    ]

,JOURNEY_ID,STOP_ID,EVENT,EVENT_TYPE,DATETIME,CARD_ID,DATETIME_NEXT,EVENT_TYPE_NEXT,TRAVEL_TIME
2,000c6c67ebae4c8bb1521cdf4296e0fd342796077e852a...,1026,BOARDED,ORIGIN,1713725709,f73b58ec-a10e-b58a-03f4-6a65e93e22b8,1.713727e+09,DESTINATION,950.0
8,002aa12d6baccd2a95fa97ec8b79071f772acfde9633d2...,1026,BOARDED,ORIGIN,1713188397,f5bee914-c7bc-a842-877b-65292500742a,1.713189e+09,DESTINATION,911.0
17,00815d32ab562013438d8a9837848e646fb565f450d893...,13597,BOARDED,ORIGIN,1712257303,e5da14ab-cc27-708f-1628-db4ac83430e5,1.712257e+09,DESTINATION,23.0
26,008fe8ea146372c6b9dfcf436d87cb2fc55aa87eb703f2...,5962,BOARDED,ORIGIN,1713878143,654442fc-18af-9658-9adf-16acce863375,1.713878e+09,DESTINATION,86.0
56,01169346d5bb5034318429c358fd83c0ee1bf18dde94c8...,5890,BOARDED,ORIGIN,1712866825,ec663ee5-57aa-59af-5291-bdb095b72b68,1.712867e+09,DESTINATION,87.0
...,...,...,...,...,...,...,...,...,...
11844,ff2cada632513b9e0a1b7daec93b7331319f676dbcfd57...,13597,BOARDED,ORIGIN,1714370628,6175da2c-f1f0-29ce-dbb5-67e46bd33d32,1.714371e+09,DESTINATION,714.0
11850,ff3122ca10a0323feda9383ee5c3f31fdf39983f92f296...,1026,BOARDED,ORIGIN,1713363555,330f50da-1f01-4cd4-19d3-c45d55bd69cb,1.713364e+09,DESTINATION,282.0
11856,ff7c64adcab02a6ea23600eab58a8e15472d383e20cc7a...,2175,BOARDED,ORIGIN,1713944874,eaeb6fb9-62c9-11f7-25e9-6698cf22d88b,1.713945e+09,DESTINATION,127.0
11863,ff86b3f824135dc852d4d1ab9af1b456b29f3842015f10...,5929,BOARDED,ORIGIN,1712074886,886cae2d-a071-8c20-9b4c-cea81ea4c690,1.712075e+09,DESTINATION,87.0
